<a href="https://colab.research.google.com/github/marcexpositg/CRISPRed/blob/master/02.Model/2.4.Featurization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from Bio import SeqIO
from math import exp
from re import findall
from Bio.SeqUtils import MeltingTemp as mt
import csv
from itertools import product

def read_target_seqs(target_seqs_file):
    col_names = ["gRNA_id", "target_seq"]
    target_seqs = pd.read_csv(target_seqs_file, names = col_names)
    return target_seqs

#############
## Strategy inespecific sequence features
#############

def onehotencoder(seq, nt_num):
    '''Convert to single and di-nucleotide hotencode. Adapted from Lindel'''
    seq = str(seq).upper()
    nt = ['A','C','G','T']
    head = []
    l = len(seq)
    # All nucleotide options per position (A0,C0,G0,T0,A1,C1,G1,T1...):
    # In the case of single nucleotide encoding
    if nt_num == 1:
        for k in range(l):
            for i in range(4):
                head.append(nt[i]+str(k))
    # For dinucleotides
    elif nt_num == 2:
        for k in range(l-1):
            for i in range(4):
                for j in range(4):
                    head.append(nt[i]+nt[j]+str(k))
    # For trinucleotides
    elif nt_num == 3:
        for k in range(l-2):
            for i in range(4):
                for j in range(4):
                    for h in range(4):
                        head.append(nt[i]+nt[j]+nt[h]+str(k))
    # Encoding
    head_idx = {}
    for idx, key in enumerate(head):
        # Dictionary of classes per position 0:A0, 1:C0
        head_idx[key] = idx
    # Initialize numpy array of seq_len*4 for single nt or seq_len-1*4*4 for di-nucleotides
    encode = np.zeros(len(head_idx))
    # Encode
    if nt_num == 1:
        for j in range(l):
            # nt in the sequence + position in the sequence to get the dictionary key o codified nts
            encode[head_idx[seq[j]+str(j)]] = 1.
    elif nt_num == 2:
        # For dints the last nt will be alone
        for k in range(l-1):
            encode[head_idx[seq[k:k+2]+str(k)]] = 1.
    elif nt_num == 3:
        # For dints the last nt will be alone
        for m in range(l-2):
            encode[head_idx[seq[m:m+3]+str(m)]] = 1.
    return(encode, head)


def seq_pattern_count(seq, nts):
    """Count the numper of each di-, tri-, ..., nucleotide.
        Position independent combinations of nucleotides """
    tri_nts = product('ACGT', repeat=nts)
    count_arr = list()
    header = list()
    for i,j in enumerate(tri_nts):
        tnt = ''.join(j)
        header.append(tnt)
        count_arr.append(seq.count(tnt))
    return (count_arr, header)


def GCcontent(seq):
    ''' Percent of GC in a given sequence'''
    seq = str(seq).upper()
    percent = ((seq.count('G')+seq.count('C'))/len(seq))*100
    return percent


def homopolymer(seq):
    """ 1 for sequences with homopolymers. At least 4 nts"""
    seq = str(seq).upper()
    nt = ['A', 'C', 'G', 'T']
    encode = np.zeros(len(nt))
    for i in range(0, len(nt)):
        if nt[i]*4 in seq:
            encode[i] = 1.
    return (encode, nt)

#### RNA/DNA hybriditzation Tm ((From biopython)
# Sugimoto et al. (1995), Biochemistry 34: 11211-11216
#
#R_DNA_NN1 = {
#"init": (1.9, -3.9), "init_A/T": (0, 0), "init_G/C": (0, 0),
#"init_oneG/C": (0, 0), "init_allA/T": (0, 0), "init_5T/A": (0, 0),
#"sym": (0, 0),
#"AA/TT": (-11.5, -36.4), "AC/TG": (-7.8, -21.6), "AG/TC": (-7.0, -19.7),
#"AT/TA": (-8.3, -23.9), "CA/GT": (-10.4, -28.4), "CC/GG": (-12.8, -31.9),
#"CG/GC": (-16.3, -47.1), "CT/GA": (-9.1, -23.5), "GA/CT": (-8.6, -22.9),
#"GC/CG": (-8.0, -17.1), "GG/CC": (-9.3, -23.2), "GT/CA": (-5.9, -12.3),
#"TA/AT": (-7.8, -23.2), "TC/AG": (-5.5, -13.5), "TG/AC": (-9.0, -26.1),
#"TT/AA": (-7.8, -21.9)}

#### RNA folding energy --> sapacer or spacer+scaffold
# Cluster installed version of mfold web application has been used to precompute the free energy of the most
# probable secundary structure
# M. Zuker
# Mfold web server for nucleic acid folding and hybridization prediction.
# Nucleic Acids Res. 31 (13), 3406-15, (2003)



#############
## Base editing specific sequence features
#############
def editable(seq, base):
    seq = str(seq).upper()
    base = base.upper()
    encode = np.zeros(len(seq))
    for pos,i in enumerate(seq):
        if i == base:
            encode[pos] = 1.
    return encode


#############
## DBS repair specific sequence features
#############
def microhomology(seq, cut_site=30):
    '''
    Adapted from Sangsu Bae, 2014 (Microhomology-based choice of Cas9 nuclease target sites)

    This function allows the calculation of the microhomology score. It takes into account the distance and the length of
    the sequence homology.
    '''
    # Capitalize and use as a string the sequence:
    seq = str(seq).upper()
    # Length weight
    length_weight = 20.0
    # Cut sides length
    left = cut_site  # Insert the position expected to be broken.
    right = len(seq) - int(left)
    # variable declaration
    k = int()  # sequence size to be used for checking homology
    i = int()  # start position of sequence substraction at the left side of the cut
    j = int()  # start position of sequence substraction at the right side of the cut

    ### PARSING SEQUENCE IN RESEARCH OF MICROHOMOLOGY SEQUENCES BETWEEN BOTH SIDES OF THE CUT ###
    microhomo = list()
    for k in range(2, left)[::-1]:  # From length to 2, 1 by 1. So, from bigger homological segments to smaller
        for j in range(left, left + right - k + 1):
            for i in range(0, left - k + 1):
                # comparing pice of left side of the cut sequence with a pice of the right side of the cut
                if seq[i:i + k] == seq[j:j + k]:
                    delLen = j - i  # deletion length
                    # Homology sequence, start and end points for both sides and deletion size are saved in a list of lists
                    microhomo.append([seq[i:i + k], i, i + k, j, j + k, delLen])

                    ### REMOVING DUPLICATIONS ###
    # After searching out all microhomology patterns, duplication should be removed!!
    trimed = list()

    # Check if there is content in the list and inisialize the counters
    if len(microhomo) != 0:
        sum_score_3 = 0
        sum_score_not_3 = 0
        scoreSummary = list()

        # Access to the info (6 fields) inside each one of the hits
        for i in range(len(microhomo)):
            n = 0
            score_3 = 0
            score_not_3 = 0

            left_start = microhomo[i][1]
            left_end = microhomo[i][2]
            right_start = microhomo[i][3]
            right_end = microhomo[i][4]

            # Access to the info again to compare. In the first case there is no reference, but in the second case
            # the first will be retrived, and in the third case the first and the second will be used as references.
            for j in range(i):

                left_start_ref = microhomo[j][1]
                left_end_ref = microhomo[j][2]
                right_start_ref = microhomo[j][3]
                right_end_ref = microhomo[j][4]

                # Each homoloy pattern is comparet with those bigger. If it is inside a bigger one in both sides,
                #  it will not be token into account
                if (left_start >= left_start_ref) and (left_end <= left_end_ref) and (
                            right_start >= right_start_ref) and (right_end <= right_end_ref):
                    if (left_start - left_start_ref) == (right_start - right_start_ref) and (
                                left_end - left_end_ref) == (right_end - right_end_ref):
                        n += 1
                        break
                else:
                    pass

                    # From all hits duplicated, the bigger one is the one saved
            if n == 0:

                # Save all the info of the trimed microhomology events
                trimed.append(microhomo[i])

                # Let's calculate the score!!
                length = microhomo[i][5]
                pattern = microhomo[i][0]

                length_factor = round(1 / exp((length) / (length_weight)), 3)
                num_GC = len(findall('G', pattern)) + len(findall('C', pattern))
                score = 100 * length_factor * ((len(pattern) - num_GC) + (num_GC * 2))

                # This gives the image of how the outcome will looks like, the pattern that is homologous, length of the deletion,
                # score of the microhomology deletion and start point of the delation.
                scoreSummary.append([seq[0:left_end] + '-' * length + seq[right_end:], pattern, length, round(score, 1),
                                     len(seq[0:left_end])])

                # Sum score in different counters in function of out or in frame deletion
                if (length % 3) == 0:
                    sum_score_3 += score
                else:
                    sum_score_not_3 += score

        microhom_score = sum_score_3 + sum_score_not_3
        # outFrame_score = (sum_score_not_3)*100/(sum_score_3+sum_score_not_3)

        return (scoreSummary, round(microhom_score, 1))


if __name__ == '__main__':
    #######
    ## Load sequences to compute the features
    #######
    #data_path = './Data/Library/'
    #Spacer + PAM
    #fasta_gRNAsequences = data_path+'./C3H_gRNAs.fasta'
    #fasta_gRNAsequences = data_path+'./C3H_targets.csv'
    fasta_gRNAsequences = './C3H_targets.csv'
    ### Explored range of the sequence
    #ini = 0
    #end = 20
    cutsite = 60
    ini = cutsite - 17
    end = cutsite + 3

    # Import sequences as pd data.frame
    target_seqs_data = read_target_seqs(fasta_gRNAsequences)

    ######
    ### Lists of features to get (this is the list of general features that can be used for any strategy)
    ######
    ids = list()
    ntdtrt = list() # Nucleotides, dinucleotides and trinucleotides per position one hot encoded
    GC_content = list()
    nucleotides_count = list()
    #dinucleotides_count = list()
    #trinucleotides_count = list()
    #homopolymer_list = list()
    #hybriditzations = list()
    #hybriditzations_seed = list()
    #secundary_structures = list()
    #secundary_structures_spacer = list()

    # ######
    # ### Load pre-computed features (secundary structures)
    # ######
    # # Dictionary with free energy of secundary structures
    # features_path = './Data/Features/'
    # mfold_path = features_path+'freeEnergy_spacer-scaffold.csv'
    # fold_dict = dict()
    # with open(mfold_path) as csv_file:
    #     csv_reader = csv.reader(csv_file, delimiter=',')
    #     for row in csv_reader:
    #         fold_dict[row[0]] = row[1]
    # # The same but for just the spacer
    # mfold_spacer_path = features_path+'./freeEnergy_spacer.csv'
    # fold_spacer_dict = dict()
    # with open(mfold_spacer_path) as csv_file:
    #     csv_reader = csv.reader(csv_file, delimiter=',')
    #     for row in csv_reader:
    #         fold_spacer_dict[row[0]] = row[1]
    #
    #######
    ### Calculate general features
    #######
    # if starting with a FASTA list of targets
    #for record in SeqIO.parse(fasta_gRNAsequences,'fasta'):
    # if starting with the data.frame file
    for seq_id, seq_nt in target_seqs_data.iterrows():
        # Ids
        ids.append(seq_nt["gRNA_id"])
        # Nucleotidic, dinucleotidic and trinucleotidic one hot encoded sequences
        gRNA_nt_hotencode = onehotencoder(seq_nt["target_seq"][ini:end], 1)[0]
        gRNA_dint_hotencode = onehotencoder(seq_nt["target_seq"][ini:end], 2)[0]
        #gRNA_trint_hotencode = onehotencoder(record.seq[ini:end], 3)[0]
        #ntdtrt_encoded = np.concatenate((gRNA_nt_hotencode, gRNA_dint_hotencode, gRNA_trint_hotencode))
        ntdtrt_encoded = np.concatenate((gRNA_nt_hotencode, gRNA_dint_hotencode))
        ntdtrt.append(ntdtrt_encoded)
        # Nucleotide, dinucleotide and trinucleotide count independent to the position
        nts = seq_pattern_count(seq_nt["target_seq"][ini:end], 1)[0]
        nucleotides_count.append(nts)
        dints = seq_pattern_count(seq_nt["target_seq"][ini:end], 2)[0]
        #dinucleotides_count.append(dints)
    #     trints = seq_pattern_count(record.seq[ini:end], 3)[0]
    #     trinucleotides_count.append(trints)
    #     # Homopolymers (min 4nt)
    #     homop = homopolymer(record.seq[ini:end])[0]
    #     homopolymer_list.append(homop)
        # GC content
        GC_content.append(GCcontent(seq_nt["target_seq"][ini:end]))
    #     # Hibriditzation
    #     tm = mt.Tm_NN(record.seq[ini:end], nn_table=mt.R_DNA_NN1)
    #     hybriditzations.append(tm)
    #     tm_seed = mt.Tm_NN(record.seq[10:20], nn_table=mt.R_DNA_NN1)
    #     hybriditzations_seed.append(tm_seed)
    #     # Secundary structures
    #     secundary_structures.append(fold_dict[record.id])
    #     secundary_structures_spacer.append(fold_spacer_dict[record.id])
    #     # Editability
    #     #gRNA_editable_C = editable(record.seq[0:20], 'C')
    #     #gRNA_editable_A = editable(record.seq[0:20], 'A')

    # Headers to identify sequence features
    nt_head = onehotencoder(target_seqs_data.iloc[0,1][ini:end],1)[1]
    dnt_head = onehotencoder(target_seqs_data.iloc[0,1][ini:end],2)[1]
    # trint_head = onehotencoder(record.seq[ini:end],3)[1]
    # homo_nts = homopolymer(record.seq[ini:end])[1]
    # homopolymer_head = ['poly_' + s for s in homo_nts]
    nt_count_head = seq_pattern_count(target_seqs_data.iloc[0,1][ini:end],1)[1]
    dint_count_head = seq_pattern_count(target_seqs_data.iloc[0,1][ini:end],2)[1]
    # trint_count_head = seq_pattern_count(record.seq[ini:end],3)[1]

    #######
    ### Calculate MMEJ features
    #######
    #MH_fasta = './C3H_cutSite_70.fa'
    #for record in SeqIO.parse(MH_fasta,'fasta'):
    #    possible_MH = microhomology(record.seq, 35) #Tpple:
        # 0 --> MHs [seq_result, patttern, del_length, MH_score and del_start
        # 1 --> total MH score (sum of all possibl MH deletions


    ########
    ### Generate matrix and file with all selected features
    ########
    ### Data frame with all together (1612x396): features and ids
    df0 = pd.DataFrame(ids, columns=['ids'])
    # df1 = pd.DataFrame(np.row_stack(ntdtrt), columns=nt_head+dnt_head+trint_head)
    df1 = pd.DataFrame(np.row_stack(ntdtrt), columns=nt_head + dnt_head)
    df2 = pd.DataFrame(np.row_stack(nucleotides_count), columns=nt_count_head)
    #df3 = pd.DataFrame(np.row_stack(dinucleotides_count), columns=dint_count_head)
    #df4 = pd.DataFrame(np.row_stack(trinucleotides_count), columns=trint_count_head)
    #df5 = pd.DataFrame(np.row_stack(homopolymer_list), columns=homopolymer_head)
    df = pd.concat([df0, df1, df2], axis=1, sort=False)
    #df = pd.concat([df0, df1, df2, df3, df4, df5], axis=1, sort=False)
    df['GC_content'] = GC_content
    #df['hybriditzation'] = hybriditzations
    #df['hybriditzation_seed'] = hybriditzations_seed
    #df['sec_structure'] = secundary_structures
    #df['sec_structure_spacer'] = secundary_structures_spacer
    print(df.shape)

    # csv
    out_csv = './gRNA_features.csv'
    df.to_csv(out_csv, sep=',', index=False)


    ##### DATA
    # Files: gRNA coordinates, gRNAs sequences, extended gRNA sequences (mm10 and C3H)
    # From gRNA coordinates to bed with cut site in the midel and n arm length + from bed to fasta (to extend sequences)
    ##### FEATURES
    # Nucleotides, dinucleotides, trinucleotides (one hot encode)
    # Nucleotides, dinucleotides, trinucleotides count
    # Homopolymer by nt
    # GC content
    # RNA/DNA hybriditzation (tm) [whole spacer or seed]
    # Secundary structures (mfold) [spacer or spacer+scaffold]
    ##### FEATURES (specifics for a certain strategy)
    # MH patterns
    # Editability
    ##### Metadata: DNasaI (chromatin), histone modifications...
